In [240]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [241]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [242]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [243]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34"
]

def skip_exp(filename):
    # # check sonnet vs haiku pred
    # if 'v11' in filename: return True
    # if ('0830' in filename and 'prompt2' in filename): return False
    # if ('sonnetpred' in filename): return False
    # return True

    # # check prompts
    # if 'haikupred' not in filename: return True
    # # if 'v12' in filename and 'prompt2' in filename: return False
    # if 'v11' in filename: 
    #     if 'prompt2' in filename: return False
    #     if 'prompt3' in filename: return False
    #     if 'prompt4' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # # check runs
    # if 'haikupred' not in filename: return True
    # if '0831' in filename and 'prompt3' in filename: return False
    # if 'v11' in filename:
    #     if 'prompt3' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # check known test
    if 'v13' not in filename and 'v16' not in filename: return True


def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    # segs = ['history', 'demo', 'persona', 'v11', 'v10', 'prompt3', 'prompt4', 'run1', 'run2', 'run3']
    segs = ['history', 'demo', 'persona', 'v16', 'v13', '500', '20', '40', 'withname', 'all', '200', '80', 'allq', '2000', '5']
    for train in ['200', '500', '2000']: segs.append('train'+train)
    for top in ['20', '40', '80']: segs.append('top'+top)
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    # print(filename, '->', final)
    return final

# Main

In [244]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name):
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]


    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    acc_df.sort_values(by='acc', ascending=False)


    test_q_keys = test_q_keys_mapping[survey_name]
    print(test_q_keys)
    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
    sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue

        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = test_q_keys[q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [245]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    merged = get_full_results(survey_name)
    display(merged)
    print('\n\n\n')

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ba25ea7fca0>>
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/pump/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


American_Trends_Panel_W34
['SCI1_W34', 'EAT1_W34', 'FUD35_W34', 'MED2G_W34', 'MED6D_W34', 'MED6E_W34']
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |about half of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |about half of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_history_demo,'=910/1518,0.599473,"{0.0: 910, 2.0: 99, 1.0: 483, 3.0: 15}",0.481752,3,8
v13_history,'=906/1518,0.596838,"{0.0: 906, 1.0: 461, 2.0: 123, 3.0: 19}",0.506296,1,8
v16_500_40_withname_history_demo_persona,'=869/1518,0.572464,"{0.0: 869, 1.0: 523, 2.0: 104, 3.0: 14}",0.511921,0,8
v16_2000_40_withname_history_demo_persona,'=868/1518,0.571805,"{0.0: 868, 1.0: 534, 2.0: 94, 3.0: 14}",0.50596,0,8
v16_500_40_withname_all_history_demo_persona,'=866/1518,0.570487,"{0.0: 866, 1.0: 535, 2.0: 97, 3.0: 12}",0.506623,0,8
v16_500_80_withname_history_demo_persona,'=859/1518,0.565876,"{0.0: 859, 1.0: 545, 2.0: 94, 3.0: 12}",0.509272,0,8
v16_train2000_top40_skew3_withname_history_demo_persona,'=856/1518,0.5639,"{0.0: 856, 1.0: 545, 2.0: 95, 3.0: 14}",0.51457,0,8
v16_2000_80_withname_history_demo_persona,'=855/1518,0.563241,"{0.0: 855, 1.0: 539, 2.0: 100, 3.0: 16}",0.521192,0,8
v16_persona_history_demo_persona,'=853/1518,0.561924,"{0.0: 853, 1.0: 548, 2.0: 95, 3.0: 14}",0.516556,0,8
v16_2000_40_5_withname_history_demo_persona,'=852/1518,0.561265,"{1.0: 544, 0.0: 852, 2.0: 100, 3.0: 14}",0.52053,0,8


In [ ]:
857

857